In [124]:
from math import cos, asin, sqrt, pi
import json
import deap

In [125]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from joblib import Parallel, delayed


def _distance(location1, location2):
    lat1, lon1 = location1
    lat2, lon2 = location2
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...


def _initIndividual(pcls, lengths, total):
    part = pcls(np.random.choice(total, lengths, replace=False))
    return part


def _mutIndividual(individual, up, indpb):
    for i in range(len(individual)):
        rn = random.random()
        if rn < indpb:
            nb = random.randint(0, up-1)
            if nb in individual:
                index = individual.index(nb)
                value = individual[index]
                individual[index] = individual[i]
                individual[i] = value
            else:
                individual[i] = nb
    return individual,


def _cxIndividual(ind1, ind2):
    size = len(ind1)
    start = random.randint(1, size-2)
    end = random.randint(start+1, size-1)
    children1 = [None] * start + ind1[start:end] + [None] * (size - end)
    children2 = [None] * start + ind2[start:end] + [None] * (size - end)
    to_fill = [_ for _ in range(end, size)] + [_ for _ in range(0, start)]
    ind1_order = ind1[end:] + ind1[:end]
    ind2_order = ind2[end:] + ind2[:end]
    for i in to_fill:
        for j in ind1_order:
            if j not in children2:
                children2[i] = j
                break
        for j in ind2_order:
            if j not in children1:
                children1[i] = j
                break
    for i in range(size):
        ind1[i] = children1[i]
        ind2[i] = children2[i]
    return ind1, ind2


class EvoTravel():
    def __init__(self, locations, n_population=100, n_generations=10, cxpb=0.5, mutpb=0.05, tournament_size=3, hof_size=1, verbose=False, n_jobs=-1, jobs_verbose=0, **kwargs):
        self._locations_info = locations
        self._locations = [(float(location.get('lat')), float(location.get('lng'))) for location in locations]
        self.n_population = n_population
        self.n_generations = n_generations
        self.cxpb = cxpb
        self.mutpb = mutpb
        self.tournament_size = tournament_size
        self.verbose = verbose
        self.n_jobs = n_jobs
        self.jobs_verbose = jobs_verbose
        self.hof_ = None
        self.hof_size = hof_size
        self.stats_ = None
        self.hist_ = None
        self.logbook_ = None
        self.pop_ = None
        self.generations = []

    def _nanmean(self, individuals):
        return np.nanmean([fitness for _, fitness in individuals])

    def _nanmin(self, individuals):
        return np.nanmin([fitness for _, fitness in individuals])

    def _nanmax(self, individuals):
        max_individual = max(individuals, key=lambda item: item[1])
        print('Best in gen with fitness {}: {}'.format(max_individual[1], max_individual[0]))
        self.generations.append(max_individual[0])
        return np.nanmax([fitness for _, fitness in individuals])

    def _nanstd(self, individuals):
        return np.nanstd([fitness for _, fitness in individuals])
        
    def _evaluate(self, individual, origin, destination):
        ind = list(individual)
        locations = list([origin]) + [self._locations[ind[i]] for i in range(0, len(ind))] + list([destination])
        return -sum([_distance(locations[i-1], locations[i]) for i in range(1, len(locations))]),

    def _parallel_map(self, f, *iters):
        return Parallel(n_jobs=self.n_jobs, verbose=self.jobs_verbose)(delayed(f)(list(*args)) for args in zip(*iters))

    def fit(self, travels, origin, destination):
        total = len(self._locations)
        
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMax)

        toolbox = base.Toolbox()

        # Structure initializers
        toolbox.register("individual", _initIndividual, creator.Individual, lengths=travels, total=total)

        # define the population to be a list of individuals
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # map parallel
        #toolbox.register("map", self._parallel_map)

        #----------
        # Operator registration
        #----------
        # register the goal / fitness function
        toolbox.register("evaluate", self._evaluate, origin=origin, destination=destination)

        # register the crossover operator
        toolbox.register("mate", _cxIndividual)

        # register a mutation operator with a probability
        toolbox.register("mutate", _mutIndividual, indpb=self.mutpb, up=total)

        # operator for selecting individuals for breeding the next
        # generation: each individual of the current generation
        # is replaced by the 'fittest' (best) of three individuals
        # drawn randomly from the current generation.
        toolbox.register("select", tools.selTournament, tournsize=self.tournament_size)

        pop = toolbox.population(n=self.n_population)
        hof = tools.HallOfFame(self.hof_size)

        # Stats
        stats = tools.Statistics(lambda ind: (ind, ind.fitness.values))
        stats.register("avg", self._nanmean)
        stats.register("min", self._nanmin)
        stats.register("max", self._nanmax)
        stats.register("std", self._nanstd)

        # History
        hist = tools.History()
        toolbox.decorate("mate", hist.decorator)
        toolbox.decorate("mutate", hist.decorator)
        hist.update(pop)

        pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=self.cxpb, mutpb=self.mutpb,
                                                   ngen=self.n_generations, stats=stats,
                                                   halloffame=hof, verbose=self.verbose)

        if self.verbose:
            print("Best individual has fitness: %s and params: %s" % (hof[0].fitness.values, hof[0]))
            
        self.best = hof[0]
        self.hof_ = hof
        self.stats_ = stats
        self.hist_ = hist
        self.logbook_ = logbook
        self.pop_ = pop

In [127]:
with open('mexico.json') as json_file:
    data = json.load(json_file)

In [128]:
origin = [20.6538687, -87.1067159]  # playa de carmen

In [129]:
data = data[:3] + data[4:]

In [130]:
len(data)

23

In [133]:
travel = EvoTravel(data[1:], n_population=50000, n_generations=200, mutpb=0.1, verbose=True)

In [ ]:
travel.fit(travels=22, origin=origin, destination=origin)

Best in gen with fitness (-15274.986889143269,): [4, 2, 7, 0, 3, 15, 13, 20, 10, 21, 18, 14, 16, 19, 8, 12, 5, 9, 1, 11, 6, 17]
gen	nevals	avg     	min     	max   	std    
0  	50000 	-26795.6	-37229.9	-15275	2887.71
Best in gen with fitness (-13172.930961497172,): [3, 15, 13, 20, 10, 21, 18, 14, 16, 19, 8, 12, 5, 7, 6, 0, 11, 1, 2, 9, 17, 4]
1  	27525 	-24733.7	-36147  	-13172.9	2534.23
Best in gen with fitness (-13172.930961497172,): [3, 15, 13, 20, 10, 21, 18, 14, 16, 19, 8, 12, 5, 7, 6, 0, 11, 1, 2, 9, 17, 4]
2  	27457 	-23230.1	-35994.8	-13172.9	2412.9 
Best in gen with fitness (-12687.548477137414,): [17, 15, 13, 20, 10, 21, 18, 14, 16, 19, 8, 12, 5, 7, 6, 0, 11, 1, 4, 3, 2, 9]
3  	27577 	-22015.4	-33970.1	-12687.5	2374.86
Best in gen with fitness (-12145.565892018496,): [0, 6, 11, 17, 2, 9, 3, 4, 7, 13, 14, 10, 18, 21, 5, 19, 20, 8, 16, 12, 15, 1]
4  	27493 	-21018.6	-34755.6	-12145.6	2386.77
Best in gen with fitness (-12213.300017608668,): [3, 15, 19, 7, 8, 12, 13, 20, 10, 21, 1

Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
48 	27639 	-9112.33	-34270.9	-8202.3 	2493.26
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
49 	27746 	-9023.19	-29770  	-8202.3 	2438.48
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
50 	27473 	-8930.89	-31259.9	-8202.3 	2363.06
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
51 	27217 	-8847.33	-30519.9	-8202.3 	2312.68
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
52 	27503 	-8811.97	-33522.1	-8202.3 	2307.53
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
53 	2751

Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
96 	27347 	-8812.86	-30018.4	-8202.3 	2315.06
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
97 	27468 	-8805.64	-32825.5	-8202.3 	2290.91
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
98 	27465 	-8796.35	-32132.5	-8202.3 	2277.26
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
99 	27398 	-8807.37	-30255.6	-8202.3 	2300.72
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
100	27599 	-8803.98	-30154.9	-8202.3 	2282.29
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
101	2761

Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
144	27502 	-8780.67	-30313.9	-8202.3 	2232.96
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
145	27449 	-8797.15	-29789.3	-8202.3 	2271.11
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
146	27505 	-8782.65	-32036.4	-8202.3 	2257.62
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
147	27464 	-8798.94	-30036.4	-8202.3 	2270.12
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
148	27463 	-8805.9 	-31416.1	-8202.3 	2295.53
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
149	2741

Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
192	27760 	-8810.42	-31192.2	-8202.3 	2288.16
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
193	27294 	-8788.31	-31144.7	-8202.3 	2269.38
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
194	27502 	-8803.34	-30895.5	-8202.3 	2280.64
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
195	27388 	-8789.44	-32986.4	-8202.3 	2246.03
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
196	27454 	-8809.47	-30584.1	-8202.3 	2296.13
Best in gen with fitness (-8202.299374285329,): [21, 18, 10, 14, 20, 13, 16, 8, 7, 5, 12, 19, 15, 17, 9, 11, 6, 2, 3, 1, 4, 0]
197	2739

In [111]:
for i in travel.best:
    print(str(travel._locations_info[i]) + '\n')

{'city': 'Tulum', 'lat': '19.7242834', 'lng': '-87.6788368'}

{'city': 'Sian Kaan', 'lat': '19.4980309', 'lng': '-87.7245672'}

{'city': 'Cozumel', 'lat': '20.2203773', 'lng': '-87.8405077'}

{'city': 'Chichén-Itzá', 'lat': '20.6787867', 'lng': '-88.5706656'}

{'city': 'Isla mujeres', 'lat': '21.2439809', 'lng': '-86.798255'}

{'city': 'Cancun', 'lat': '21.1214886', 'lng': '-86.9192733'}



In [112]:
travels = travel.generations + [travel.best]

In [113]:
origin

[20.6538687, -87.1067159]

In [120]:
import folium

trvl = travels[-1]
sample_coords = [origin] + [[float(travel._locations_info[i]['lat']), float(travel._locations_info[i]['lng'])] for i in trvl] + [origin]

# Build map 
map_nyc = folium.Map(location=(19.3910038,-99.2836969), zoom_start=3, 
tiles='cartodbpositron', width=270, height=480, zoom_control=False)

# Plot coordinates using comprehension list
[folium.CircleMarker(sample_coords[i], radius=1,
                color='#0080bb', fill_color='#0080bb').add_to(map_nyc) 
for i in range(len(sample_coords))]

#add lines
folium.PolyLine(sample_coords, color="#0080bb", weight=2.5, opacity=1).add_to(map_nyc)

In [121]:
map_nyc

In [116]:
import folium
import datetime as dt
import random as rnd

import os
import time
from selenium import webdriver

for idx in list(range(0, len(travels), 10)) + [len(travels)-1]:
    trvl = travels[idx]
    sample_coords = [origin] + [[float(travel._locations_info[i]['lat']), float(travel._locations_info[i]['lng'])] for i in trvl] + [origin]

    # Build map 
    map_nyc = folium.Map(location=(19.3910038,-99.2836969), zoom_start=3, 
    tiles='cartodbpositron', width=270, height=480, zoom_control=False)

    # Plot coordinates using comprehension list
    [folium.CircleMarker(sample_coords[i], radius=1,
                    color='#0080bb', fill_color='#0080bb').add_to(map_nyc) 
    for i in range(len(sample_coords))]

    #add lines
    folium.PolyLine(sample_coords, color="#0080bb", weight=2.5, opacity=1).add_to(map_nyc)
    
    # save html and png
    path = 'file:///Users/rodrigoazevedosantos/rodrigo/evolutionary_traveling_salesman'
    #os.chdir(path)

    delay=5
    map_nyc.save('testmap.html')

    fn='testmap.html'
    tmpurl='file:///{path}/{mapfile}'.format(path=os.getcwd(), mapfile=fn)

    browser = webdriver.Chrome()
    browser.get(tmpurl)
    #Give the map tiles some time to load
    time.sleep(delay)
    browser.save_screenshot('map.png')
    browser.quit()
    
    # crop and save
    from PIL import Image
    image = Image.open('map.png')
    box = (0, 0, 2*270, 2*480-60)
    cropped_image = image.crop(box)
    cropped_image.save('cropped/{}.png'.format(idx))

In [117]:
# Create Gif and remove each .png file
from pathlib import Path
import imageio

image_path = Path('cropped/')

#images = list(image_path.glob('*.png'))
image_list = []

for file_name in list(range(0, len(travels), 10)) + [len(travels)-1]:
    image_list.append(imageio.imread("cropped/{}.png".format(file_name)))
    #os.remove(file_name)
    
image_list = image_list + image_list[-1:] * 10
    
imageio.mimwrite('GifMap.gif', image_list, fps=2)

/tmp/ipykernel_42609/5859867.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image_list.append(imageio.imread("cropped/{}.png".format(file_name)))
